In [2]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output
import webbrowser

# Load the Excel file
file_path = 'Final CET.xlsx'
df = pd.read_excel(file_path)

# Function to modify seat type conditionally
def clean_seat_type(seat_type):
    categories_to_modify = ['OPEN', 'OBC', 'NT1', 'NT2', 'NT3', 'VJ', 'SC', 'ST']
    if len(seat_type) > 2 and seat_type[1:-1] in categories_to_modify:
        return seat_type[1:-1]  # Slice the seat type
    return seat_type

# Apply the function to modify the 'SEAT TYPE' column conditionally
df['MODIFIED SEAT TYPE'] = df['SEAT TYPE'].apply(clean_seat_type)

# Get unique values for dropdowns using the modified seat type
seat_type_options = ['All'] + df['MODIFIED SEAT TYPE'].unique().tolist()
branch_options = ['All'] + df['BRANCH NAME'].unique().tolist()
district_options = ['All'] + df['DISTRICT'].unique().tolist()

# Create widgets for multi-selection
seat_type_checkboxes = widgets.SelectMultiple(
    options=seat_type_options,
    description='Category:',
    disabled=False,
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='50%', height='100px')
)

branch_checkboxes = widgets.SelectMultiple(
    options=branch_options,
    description='Branch:',
    disabled=False,
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='50%', height='100px')
)

district_checkboxes = widgets.SelectMultiple(
    options=district_options,
    description='District:',
    disabled=False,
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='50%', height='100px')
)

rank_input = widgets.IntText(
    value=7000,
    description='Rank:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='50%')
)

search_button = widgets.Button(
    description='Search',
    disabled=False,
    button_style='success',
    tooltip='Search for Colleges',
    icon='search',
    layout=widgets.Layout(width='150px', height='40px')
)

output = widgets.Output()

def search_colleges(df, rank=None, seat_types=None, branches=None, districts=None, range_=300):
    if seat_types and 'All' not in seat_types:
        df = df[df['MODIFIED SEAT TYPE'].isin(seat_types)]
    
    if branches and 'All' not in branches:
        df = df[df['BRANCH NAME'].isin(branches)]
    
    if districts and 'All' not in districts:
        df = df[df['DISTRICT'].isin(districts)]
    
    initial_range = range_
    result_df = df[(df['CUT OFF (RANK)'] >= rank - range_) & (df['CUT OFF (RANK)'] <= rank + range_)]
    
    while len(result_df) < 10:
        range_ += initial_range
        result_df = df[(df['CUT OFF (RANK)'] >= rank - range_) & (df['CUT OFF (RANK)'] <= rank + range_)]
        
        if range_ > df['CUT OFF (RANK)'].max() - df['CUT OFF (RANK)'].min():
            break
    
    return result_df

def search_and_display(seat_types, branches, districts, rank):
    with output:
        clear_output()
        seat_types = list(seat_types)
        branches = list(branches)
        districts = list(districts)

        result_df = search_colleges(df, rank=rank, seat_types=seat_types, branches=branches, districts=districts)
        
        if not result_df.empty:
            # Render the DataFrame as an HTML table and save it to a file
            html_result = result_df.to_html(index=False, classes='table table-striped')
            html_content = f"""
            <html>
            <head>
            <style>
                body {{ 
    font-family: Arial, sans-serif; 
    margin: 40px; 
    background-color: #FFF3E0; 
}}

table {{
    border-collapse: collapse; 
    width: 100%; 
    margin-top: 20px; 
    table-layout: auto; /* Allows table to adapt to screen size */
}}

th, td {{
    text-align: left; 
    padding: 12px; /* Increased padding for more spacing */
}}

tr:nth-child(even) {{ 
    background-color: #F6DBC3; 
}}

tr:nth-child(odd) {{ 
    background-color: #F6DBC3; 
}}

th {{
    background-color: #F6DBC3; 
    color: black; 
}}

table, th, td {{ 
    border: 2px solid #FF9703; 
}}

h2 {{
    color: #FF9703; 
}}

h1 {{ 
    color: #FF9703; 
}}

.navbar {{
    overflow: hidden;
    background-color: #FFF3E0;
    padding: 10px;
    position: fixed;
    top: 0;
    width: 100%;
    display: flex;
    justify-content: center;
    align-items: center;
}}

.navbar img {{
    height: 60px;
    margin: 10px;
}}

.content {{
    margin-top: 100px;
}}

/* Responsive design for smaller screens */
@media screen and (max-width: 768px) {{
    table, th, td {{
        font-size: 14px; /* Adjust font size for smaller screens */
        padding: 10px; /* Adjust padding for smaller screens */
    }}
    
    .navbar img {{
        height: 50px; /* Scale down logo for smaller screens */
    }}
}}

        
            </style>
            </head>
            <body>
            <div class="navbar">
                <img src="logo.png" alt="Logo"> <!-- Replace with your image file -->
            </div>
            <div class="content">
                <h1>Congratulations!</h1>
                <h2>Filtered Colleges within the Rank Range: {rank}</h2>
            {html_result}
            </div>
            </body>
            </html>
            """
            with open('output.html', 'w') as f:
                f.write(html_content)
            
            # Open the HTML file in the default web browser
            webbrowser.open('output.html')
        else:
            display(HTML("<h2 style='color:#FF0000;'>No Results Found</h2>"))

def on_search_button_clicked(b):
    search_and_display(seat_type_checkboxes.value, branch_checkboxes.value, district_checkboxes.value, rank_input.value)

search_button.on_click(on_search_button_clicked)

ui = widgets.VBox([
    widgets.HTML("<h1 style='color:#4CAF50;'>College Finder</h1>"),
    seat_type_checkboxes,
    branch_checkboxes,
    district_checkboxes,
    rank_input,
    search_button,
    output
], layout=widgets.Layout(padding='20px'))

display(ui)